Okay, das macht absolut Sinn – du willst erst auf deinem PC alles fertig entwickeln und dann später auf den Raspberry Pi übertragen. Da die externe Festplatte noch nicht bereit ist und du ohne Anaconda auf dem Pi arbeiten wirst, können wir das Ganze flexibel gestalten. Ich passe den Plan so an, dass du jetzt auf deinem PC mit einem lokalen Testordner arbeiten kannst und später alles einfach auf den Pi überträgst.

### Aktueller Plan
1. **Entwicklung auf dem PC**: Wir simulieren die externe Festplatte mit einem lokalen Ordner (z. B. `C:\test-photos` oder `/home/user/test-photos`), damit du den Code testen kannst.
2. **Ohne Anaconda auf dem Pi**: Da du auf dem Pi kein Anaconda nutzen wirst, machen wir das Setup mit purem Node.js und npm, was sowieso schon passt, da dein Projekt auf Vite und Node basiert.
3. **Übertragung später**: Ich gebe dir am Ende eine kurze Anleitung, wie du das auf den Pi bringst.

---

### Schritt-für-Schritt auf dem PC

#### 1. Testordner für Fotos erstellen
Da die Festplatte noch nicht da ist, erstelle einen Ordner mit ein paar Testfotos auf deinem PC:
- Windows: `C:\test-photos`
- Linux/Mac: `~/test-photos`
Kopiere ein paar Bilder (z. B. `foto1.jpg`, `foto2.png`) hinein.

#### 2. Backend anpassen
Ändere den Pfad in `server.js` auf deinen Testordner. Hier ist die aktualisierte Version:

```javascript
const express = require('express');
const fs = require('fs').promises;
const path = require('path');
const cors = require('cors');
const app = express();
const port = 5000;

// Pfad zu deinem Testordner (anpassen!)
const photoDir = 'C:/test-photos'; // Windows
// const photoDir = '/home/user/test-photos'; // Linux/Mac

app.use(cors()); // Für Frontend-Zugriff

app.get('/api/random-photo', async (req, res) => {
  try {
    const files = await fs.readdir(photoDir);
    const photos = files.filter(file => /\.(jpg|jpeg|png|gif)$/i.test(file));
    if (photos.length === 0) {
      return res.status(404).send('Keine Fotos gefunden');
    }
    const randomPhoto = photos[Math.floor(Math.random() * photos.length)];
    const photoPath = path.join(photoDir, randomPhoto);
    res.sendFile(photoPath);
  } catch (error) {
    console.error(error);
    res.status(500).send('Fehler beim Laden des Fotos');
  }
});

app.listen(port, () => {
  console.log(`Server läuft auf http://localhost:${port}`);
});
```

- **Pfad anpassen**: Ersetze `C:/test-photos` mit dem tatsächlichen Pfad zu deinem Testordner. Nutze Schrägstriche (`/`) auch unter Windows, da Node.js das versteht.
- **Starten**: 
  ```bash
  cd backend
  node server.js
  ```
- Teste im Browser: `http://localhost:5000/api/random-photo` sollte ein Foto anzeigen.

#### 3. Frontend prüfen
Dein `App.tsx` und `RandomPhoto.tsx` sehen schon gut aus. Stelle sicher, dass `RandomPhoto.tsx` unverändert bleibt (wie im letzten Snippet). Starte das Frontend:
```bash
npm run dev
```
Öffne `http://localhost:5173/random-photo`. Wenn der Backend-Server läuft, solltest du jetzt ein zufälliges Foto aus deinem Testordner sehen.

#### 4. Entwicklung fertigstellen
- Teste die Funktionalität (z. B. Button für neues Foto).
- Füge bei Bedarf Styling oder weitere Features hinzu (z. B. Foto-Metadaten anzeigen).

---

### Übertragung auf den Raspberry Pi (später)
Wenn du bereit bist, das auf den Pi zu laden, hier eine kurze Anleitung:

1. **Node.js auf dem Pi installieren**:
   ```bash
   sudo apt update
   sudo apt install nodejs npm
   ```

2. **Projekt übertragen**:
   - Kopiere den gesamten Projektordner (inkl. `backend` und Frontend) auf den Pi, z. B. via USB-Stick oder `scp`:
     ```bash
     scp -r /pfad/zu/deinem/projekt pi@raspberrypi:/home/pi/
     ```
   - Alternativ: Nutze Git, falls du ein Repository hast.

3. **Festplatte mounten**:
   - Schließe die externe Festplatte an den Pi an.
   - Finde den Gerätenamen:
     ```bash
     lsblk
     ```
   - Mounten (z. B. für eine FAT32/NTFS-Platte):
     ```bash
     sudo mkdir -p /mnt/external
     sudo mount /dev/sda1 /mnt/external
     ```
   - Passe den `photoDir`-Pfad in `server.js` an (z. B. `/mnt/external/photos`).

4. **Abhängigkeiten installieren**:
   - Backend:
     ```bash
     cd /home/pi/projekt/backend
     npm install
     ```
   - Frontend:
     ```bash
     cd /home/pi/projekt
     npm install
     ```

5. **Starten**:
   - Backend: `node server.js` (im `backend`-Ordner)
   - Frontend: `npm run dev` (im Projektordner)
   - Öffne `http://localhost:5173/random-photo` auf dem Pi oder einem Gerät im Netzwerk.

6. **Autostart (optional)**:
   - Nutze `pm2`, um beide Prozesse beim Booten zu starten:
     ```bash
     sudo npm install -g pm2
     pm2 start /home/pi/projekt/backend/server.js --name "photo-backend"
     pm2 start "npm run dev" --name "photo-frontend" --cwd /home/pi/projekt
     pm2 save
     pm2 startup
     ```

---

### Nächste Schritte auf dem PC
- Erstelle den Testordner und füge ein paar Fotos hinzu.
- Starte den Backend-Server (`node server.js`).
- Teste das Frontend (`npm run dev`) und prüfe `http://localhost:5173/random-photo`.
- Wenn etwas nicht klappt, teile mir die Fehlermeldung mit!

So kannst du jetzt auf dem PC alles fertigstellen und später problemlos auf den Pi übertragen. Viel Erfolg – das wird eine coole Intranet-Seite!